In [1]:
import numpy as np
import pandas as pd
from datetime import datetime,date,time
from dateutil.relativedelta import relativedelta

import missingno as msno


### **A) Coleta dos Dados de Vento Verificado no Patio 1**

In [2]:
source_file = '../../Data/raw/eolica/01_patio_BAUET3_CAETITÉ_3_Ven_Verif_Consis.txt'

In [3]:
dfvento_consit = pd.read_csv(source_file, delimiter = ';' , header=None )

### **B) Sobre os Dados no Patio 1**

#### <span style="color:#DC143C">**A1.Formato dos Dados**

In [4]:
dfvento_consit.shape # formato da matriz

(3153, 49)

#### <span style="color:#DC143C">**A2.Visao do Dataframe**

In [5]:
dfvento_consit.head(2)  # visão de 5 linhas

,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
0,20120101,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,...,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0
1,20120102,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,...,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0


### **C) Extração os Dados de 2017**

#### <span style="color:#DC143C"> **C1. Converter a primeira coluna em Data**

In [6]:
dfvento_consit[0]= pd.to_datetime(dfvento_consit[0], format='%Y%m%d')

#### <span style="color:#DC143C"> **C2. Extração ....**

In [7]:
Linha  =  dfvento_consit.shape[0]
Coluna =  dfvento_consit.shape[1]

aCols = np.arange(0,Coluna) 
df_alvo = pd.DataFrame(columns = aCols )

data1 = datetime(2017, 1, 1)
data12 = datetime(2017, 12, 31)

for lin in range(Linha): 
    
    # 01-01-2027  
    if ( dfvento_consit.loc[lin,0] == data1 ): 
       
        variavel = dfvento_consit.values[lin,]        
        df_alvo.loc[len(df_alvo)] = variavel

        umDia = relativedelta(days=+1)
        data1 = data1 + umDia   
        
    # 31-12-2027    
    if ( dfvento_consit.loc[lin,0] == data12 ): 
        umDia = relativedelta(days=+2500)
        data1 = data1 + umDia 
                     
        
df_alvo['longitude'] = -42.75
df_alvo['latitude']  = -14.25

In [8]:
# df_alvo.head(5)

#### <span style="color:#DC143C"> **C4. Mudar a ordem das colunas**

In [9]:
cols = list(df_alvo)
cols = [cols[-1]] + cols[:-1]
df_alvo = df_alvo[cols]

cols = list(df_alvo)
cols = [cols[-1]] + cols[:-1]
df_alvo = df_alvo[cols]

# df_alvo.head(2)

### **D) Formatação dos Dados**

#### <span style="color:#DC143C"> **D1. Migrar os dados para o Formato**

In [10]:
frente = ['Longitude', 'Latitude', 'Data', 'Valor30_1', 'Valor30_2']  # Cabeçalho
df_result_vento_verif = pd.DataFrame(columns=frente)  # Criação de um dataframe vazio
    
Linha1    =  df_alvo.shape[0] 
Colunas1  =  df_alvo.shape[1] 

for lin2 in range(Linha1):  
    refhora = 0
    
    for col in range (Colunas1):        
        if col > 0:            
        
            if col % 2 != 0: # Se o número (col) for impar                   
               
                if col < 49:
                    Data = df_alvo.loc[lin2,0]
                    hora = time(hour=refhora, minute=0, second=0)            
                    dataCompleta = datetime.combine(Data, hora)
                
                    lin_alvo = [df_alvo.loc[lin2,'longitude'], df_alvo.loc[lin2,'latitude'], dataCompleta , df_alvo.loc[lin2,col], df_alvo.loc[lin2,col+1]] 
                  
                    df_result_vento_verif.loc[len(df_result_vento_verif)] = lin_alvo  # adding a row
                        
                refhora = refhora + 1          
                


#### <span style="color:#DC143C"> **D2. Agregar Magnitude para 1H**

In [11]:
df_result_vento_verif['Magnitude_Verif'] = (df_result_vento_verif['Valor30_1'] + df_result_vento_verif['Valor30_2']) / 2

In [12]:
df_result_vento_verif.head(2)

,Longitude,Latitude,Data,Valor30_1,Valor30_2,Magnitude_Verif
0,-42.75,-14.25,2017-01-01 00:00:00,9.710,9.920,9.815
1,-42.75,-14.25,2017-01-01 01:00:00,9.048,8.422,8.735


In [13]:
df_result_vento_verif = df_result_vento_verif.drop('Valor30_1', 1)
df_result_vento_verif = df_result_vento_verif.drop('Valor30_2', 1)

In [14]:
df_result_vento_verif.head(2)

,Longitude,Latitude,Data,Magnitude_Verif
0,-42.75,-14.25,2017-01-01 00:00:00,9.815
1,-42.75,-14.25,2017-01-01 01:00:00,8.735


### **E. Exporta Resultados**

In [15]:
output_file = '../../Data/process/result_01_patio_ventoverif.csv'
df_result_vento_verif.to_csv(output_file, sep=';', encoding='utf-8')